In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/classification-dataset/Combined_Data(3000).csv


In [2]:
!pip install transformers datasets evaluate accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.9 MB/s eta 0:00:00


In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
import pandas as pd
from datasets import DatasetDict, Dataset

# Step 1: Load CSV files
train_df = pd.read_csv('/kaggle/input/classification-dataset/Combined_Data(3000).csv')  # Replace with your train file path
test_df = pd.read_csv('/kaggle/input/classification-dataset/Combined_Data(3000).csv')    #

# Step 2: Define the label encoding
label_encoding = {"Education": 0, "Finance": 1, "Legal": 2, "Medical": 3}

# Replace the label column in both train and test DataFrames
train_df['Label'] = train_df['Label'].replace(label_encoding)
test_df['Label'] = test_df['Label'].replace(label_encoding)

# Step 3: Convert DataFrames to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)





# Step 4: Create a DatasetDict
dataset_dict = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

/tmp/ipykernel_30/3965881142.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_df['Label'] = train_df['Label'].replace(label_encoding)
/tmp/ipykernel_30/3965881142.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_df['Label'] = test_df['Label'].replace(label_encoding)


In [8]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['Sentences', 'Label'],
        num_rows: 12000
    })
    test: Dataset({
        features: ['Sentences', 'Label'],
        num_rows: 12000
    })
})

In [9]:
def preprocess_function(examples):
    return tokenizer(examples["Sentences"], truncation=True)

In [21]:
tokenized_dataset = dataset_dict.map(preprocess_function, batched=True)

# Drop the unwanted column from the training dataset
train_dataset = tokenized_dataset['train']

# Convert to pandas DataFrame, drop the column, and convert back
train_df = train_dataset.to_pandas().drop(columns=['Sentences'])
train_dataset = Dataset.from_pandas(train_df)

# Update the DatasetDict
tokenized_dataset['train'] = train_dataset





# Drop the unwanted column from the training dataset
test_dataset = tokenized_dataset['test']

# Convert to pandas DataFrame, drop the column, and convert back
test_df = test_dataset.to_pandas().drop(columns=['Sentences'])
test_dataset = Dataset.from_pandas(test_df)

# Update the DatasetDict
tokenized_dataset['test'] = test_dataset


Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

In [31]:
tokenized_dataset

# Rename the "Label" column to "label" in the training dataset
def rename_label(example):
    example['label'] = example.pop('Label')
    return example

tokenized_dataset['train'] = tokenized_dataset['train'].map(rename_label)
tokenized_dataset['test'] = tokenized_dataset['test'].map(rename_label)



Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

In [32]:
tokenized_dataset['train'][0]

{'input_ids': [101,
  2061,
  2096,
  2327,
  4177,
  1999,
  2859,
  1010,
  1999,
  1996,
  2822,
  2231,
  2024,
  5204,
  1997,
  2311,
  1010,
  2024,
  5204,
  1997,
  8915,
  2232,
  5197,
  1997,
  2311,
  1037,
  2614,
  3423,
  7705,
  2005,
  20014,
  4402,
  2571,
  6593,
  8787,
  3200,
  3860,
  1010,
  7285,
  2320,
  2153,
  1010,
  2926,
  2012,
  1996,
  2334,
  2504,
  1010,
  2038,
  2000,
  2022,
  5301,
  1010,
  9839,
  1012,
  102],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'label': 0}

In [11]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [12]:
import evaluate

accuracy = evaluate.load("accuracy")

In [13]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [14]:
id2label = {0: "Education", 1: "Finance", 2: "Legal", 3:"Medical"}
label2id = {"Education": 0, "Finance": 1, "Legal":2, "Medical":3}

In [33]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=4, id2label=id2label, label2id=label2id
)

# # # Load model directly
# # Load model directly
# from transformers import AutoTokenizer, AutoModelForMaskedLM

# # tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
# model = AutoModelForMaskedLM.from_pretrained("google-bert/bert-base-uncased")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = model.to(device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
import os
os.environ["WANDB_DISABLED"] = "false"

In [34]:
tokenized_dataset["train"]

Dataset({
    features: ['input_ids', 'attention_mask', 'label'],
    num_rows: 12000
})

In [35]:
training_args = TrainingArguments(
    output_dir="my_classifer_Model",
    learning_rate=2e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    report_to = ['none']
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator
#     compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,0.130000,0.037523
2,0.011800,0.002576


TrainOutput(global_step=1500, training_loss=0.05929474449157715, metrics={'train_runtime': 581.7653, 'train_samples_per_second': 41.254, 'train_steps_per_second': 2.578, 'total_flos': 2254551347516160.0, 'train_loss': 0.05929474449157715, 'epoch': 2.0})

In [36]:
from transformers import pipeline

text = "These free-text notes include physician and nursing notes, radiology, and pathology reports. Prior to bringing in a note type, the notes are de-identified. There are two steps in our de-identification framework: notes de-identification and notes adjudication"

classifier = pipeline("text-classification", model= "/kaggle/working/my_classifer_Model/checkpoint-1500")
classifier(text)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'Medical', 'score': 0.6921417713165283}]